In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from PIL import Image

In [8]:
num_classes = 2
learning_rate = 0.001
dropout_rate = 0.5
batch_size = 32
num_epochs = 25

In [9]:
class EfficientNetIrisTumor(nn.Module):
    def __init__(self, num_classes, dropout_rate):
        super(EfficientNetIrisTumor, self).__init__()
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.efficientnet.classifier[1].in_features, num_classes)
        )

    def forward(self, x):
        return self.efficientnet(x)

In [11]:
model = EfficientNetIrisTumor(num_classes=num_classes, dropout_rate=dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [12]:
class CustomDataset(Dataset):
    def __init__(self, folder_path, label, transform=None):
        self.folder_path = folder_path
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

In [13]:
tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/YES"
no_tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/NO"

In [14]:
tumor_dataset = CustomDataset(tumor_path, label=1, transform=transform)
no_tumor_dataset = CustomDataset(no_tumor_path, label=0, transform=transform)
combined_dataset = ConcatDataset([tumor_dataset, no_tumor_dataset])
train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_data, val_data = random_split(combined_dataset, [train_size, val_size])

In [15]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100. * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cpu'), labels.to('cpu')

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_epoch_acc = 100. * val_correct / val_total
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')



Epoch 1/25, Loss: 0.4228, Accuracy: 80.32%
Validation Loss: 0.1289, Validation Accuracy: 93.62%
Epoch 2/25, Loss: 0.1077, Accuracy: 95.21%
Validation Loss: 0.0880, Validation Accuracy: 97.87%
Epoch 3/25, Loss: 0.0418, Accuracy: 98.94%
Validation Loss: 0.0641, Validation Accuracy: 97.87%
Epoch 4/25, Loss: 0.0221, Accuracy: 99.47%
Validation Loss: 0.0008, Validation Accuracy: 100.00%
Epoch 5/25, Loss: 0.0088, Accuracy: 100.00%
Validation Loss: 0.0004, Validation Accuracy: 100.00%
Epoch 6/25, Loss: 0.0387, Accuracy: 99.47%
Validation Loss: 0.0400, Validation Accuracy: 97.87%
Epoch 7/25, Loss: 0.0171, Accuracy: 99.47%
Validation Loss: 0.4756, Validation Accuracy: 95.74%
Epoch 8/25, Loss: 0.1363, Accuracy: 97.34%
Validation Loss: 0.0651, Validation Accuracy: 97.87%
Epoch 9/25, Loss: 0.0222, Accuracy: 99.47%
Validation Loss: 0.0074, Validation Accuracy: 100.00%
Epoch 10/25, Loss: 0.0211, Accuracy: 99.47%
Validation Loss: 0.0018, Validation Accuracy: 100.00%


In [16]:
torch.save(model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Efficientnet.pth')
